# Session 2 - Parameter Calibration

Usually, the parameters we obtain from the characterization of the cell result is simulations that are off from cycling data. Therefore, before using a model, it is good practice to carry out a "Calibration" step, where we fit a small subset of the model parameters to experimental voltage profiles of the same cell.

In this session, we will calibrate the Xu 2025 parameter set to experimental voltage curves from the same cell. 

Let's import BattMo and some other packages we will use.

In [1]:
using BattMo, GLMakie, CSV, DataFrames, Jutul

### Load the experimental data

We will calibrate our model in two steps: 
1. We will adjust the stoichiometric coefficients and maximum concentrations of the active materials, to fit a cell voltage curve at C/2.
2. We will adjust the reaction rate constants and diffusion coefficients in the active materials, to fit a cell voltage curve at 2C.

We first load the datasets.

In [4]:
df_05 = CSV.read("Xu2015_data/Xu_2015_voltageCurve_05C.csv", DataFrame)
df_1 = CSV.read("Xu2015_data/Xu_2015_voltageCurve_05C.csv", DataFrame)
df_2 = CSV.read("Xu2015_data/Xu_2015_voltageCurve_05C.csv", DataFrame)

dfs = [df_05, df_1, df_2]

3-element Vector{DataFrame}:
 20×2 DataFrame
 Row │ 0          3.3861097472267683 
     │ Float64   Float64             
─────┼───────────────────────────────
   1 │  357.766              3.29433
   2 │  715.976              3.26386
   3 │ 1074.19               3.2519
   4 │ 1432.4                3.24463
   5 │ 1790.61               3.24649
   6 │ 2148.82               3.24575
   7 │ 2507.03               3.24625
   8 │ 2877.59               3.24726
  ⋮  │    ⋮               ⋮
  14 │ 5026.85               3.22731
  15 │ 5385.06               3.21844
  16 │ 5743.27               3.21746
  17 │ 6101.48               3.19921
  18 │ 6472.04               3.17788
  19 │ 6817.9                2.88079
  20 │ 7188.46               2.17905
                       5 rows omitted
 20×2 DataFrame
 Row │ 0          3.3861097472267683 
     │ Float64   Float64             
─────┼───────────────────────────────
   1 │  357.766              3.29433
   2 │  715.976              3.26386
   3 │ 1074.19   

### Run a simulation of the original parameters
Now we run a baseline simulation using the parameters obtained only from characterization of the cell. We load the parameter set, ensure an appropiate lower voltage limit and DRate, and run the simulation as we saw in previous tutorials.

In [2]:
cell_parameters = load_cell_parameters(; from_default_set = "Xu2015")
cycling_protocol = load_cycling_protocol(; from_default_set = "CCDischarge")

cycling_protocol["LowerVoltageLimit"] = 2.25
model_setup = LithiumIonBattery()

cycling_protocol["DRate"] = 0.5
sim = Simulation(model_setup, cell_parameters, cycling_protocol)
output0 = solve(sim);



✔️ Validation of ModelSettings passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CellParameters passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SimulationSettings passed: No issues found.
──────────────────────────────────────────────────
Jutul: Simulating 2 hours, 12 minutes as 163 report steps


Progress   1%|█                                          |  ETA: 0:23:03

Progress  85%|█████████████████████████████████████      |  ETA: 0:00:03

Progress 100%|███████████████████████████████████████████| Time: 0:00:18


╭────────────────┬───────────┬───────────────┬──────────╮
│ Iteration type │  Avg/step │  Avg/ministep │    Total │
│                │ 138 steps │ 138 ministeps │ (wasted) │
├────────────────┼───────────┼───────────────┼──────────┤
│ Newton         │   2.18841 │       2.18841 │  302 (0) │
│ Linearization  │   3.18841 │       3.18841 │  440 (0) │
│ Linear solver  │   2.18841 │       2.18841 │  302 (0) │
│ Precond apply  │       0.0 │           0.0 │    0 (0) │
╰────────────────┴───────────┴───────────────┴──────────╯
╭───────────────┬─────────┬────────────┬─────────╮
│ Timing type   │    Each │   Relative │   Total │
│               │      ms │ Percentage │       s │
├───────────────┼─────────┼────────────┼─────────┤
│ Properties    │  0.5119 │     0.98 % │  0.1546 │
│ Equations     │ 11.2439 │    31.24 % │  4.9473 │
│ Assembly      │  5.1733 │    14.38 % │  2.2763 │
│ Linear solve  │  0.7675 │     1.46 % │  0.2318 │
│ Linear setup  │  0.0000 │     0.00 % │  0.0000 │
│ Precond apply │  

Once the simulation completes, we can inspect the resutling voltage curves, and compare them with the experimental voltage curves.

In [ ]:
#Simulation data
time_series0 = get_output_time_series(output0)
t0 = time_series0[:Time]
V0 = time_series0[:Voltage]

#Experimental data
t_exp_05 = df_05[:,1]
V_exp_05 = df_05[:,2]
t_exp_1 = df_1[:,1]
V_exp_1 = df_1[:,2]

#Plot
fig = Figure()
ax = Axis(fig[1, 1], title = "CRate = 0.5", xlabel = "Time / s", ylabel = "Voltage / V")
lines!(ax, t0, V0, label = "Simulation 0.5C: original parameters")
lines!(ax, t_exp_05, V_exp_05, label = "Experiment 0.5C")
axislegend(position = :lb)
fig

We can see that the simulation with original parameters does not match well the experiment. Lets therefore fit some parameters to the experimental data.

### Set up the low-rate calibration

We have developed a calibration function that takes as inputs the voltage and time arrays of the data, along with the initial simulation setup.

In [11]:
calibration_05 = VoltageCalibration(t_exp_05, V_exp_05, sim)

VoltageCalibration([357.76627218934914, 715.9763313609469, 1074.1863905325445, 1432.396449704142, 1790.6065088757396, 2148.816568047337, 2507.0266272189347, 2877.5887573964496, 3223.44674556213, 3594.0088757396447, 3952.2189349112427, 4310.42899408284, 4668.639053254437, 5026.8491124260345, 5385.059171597633, 5743.2692307692305, 6101.479289940828, 6472.041420118343, 6817.899408284024, 7188.461538461537], [3.2943262673632967, 3.2638600156322126, 3.2518999695748874, 3.2446281622882482, 3.246486083133996, 3.245753135185418, 3.246253934281757, 3.2472569925301102, 3.2356583102522136, 3.2351808720466657, 3.2359284205519883, 3.237169467875278, 3.227800290612279, 3.2273140920726844, 3.2184384136276525, 3.217458716270091, 3.1992065602836877, 3.177878797019038, 2.8807910485472883, 2.179051790010771], Simulation(BattMo.run_battery, LithiumIonBattery("Setup object for a P2D lithium-ion model", {
    "RampUp" => "Sinusoidal"
    "Metadata" =>     {
        "Description" => "Default model settings f

This calibration object is a handy way to tailor the main settings needed to run a calibration: 
* Which model parameters are frozen
* Which model parameters are being fitted
* What are the minimum and maximum bounds of the parameters to be fitted
* The results of the calibration, i.e. the optimal parameters.

All paremters are forzen by default, so we now need to free those we are interested in, and apply some bounds to each to ensure they remain within expected ranges. Below, we free the stoichiometric coefficients and maximum concentrations.

In [13]:
free_calibration_parameter!(calibration_05,
    ["NegativeElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC100"];
    lower_bound = 0.0, upper_bound = 1.0)
free_calibration_parameter!(calibration_05,
    ["PositiveElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC100"];
    lower_bound = 0.0, upper_bound = 1.0)

# "StoichiometricCoefficientAtSOC0" at both electrodes
free_calibration_parameter!(calibration_05,
    ["NegativeElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC0"];
    lower_bound = 0.0, upper_bound = 1.0)
free_calibration_parameter!(calibration_05,
    ["PositiveElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC0"];
    lower_bound = 0.0, upper_bound = 1.0)

#  "MaximumConcentration" of both electrodes
free_calibration_parameter!(calibration_05,
    ["NegativeElectrode","ActiveMaterial", "MaximumConcentration"];
    lower_bound = 10000.0, upper_bound = 1e5)
free_calibration_parameter!(calibration_05,
    ["PositiveElectrode","ActiveMaterial", "MaximumConcentration"];
    lower_bound = 10000.0, upper_bound = 1e5)

VoltageCalibration([357.76627218934914, 715.9763313609469, 1074.1863905325445, 1432.396449704142, 1790.6065088757396, 2148.816568047337, 2507.0266272189347, 2877.5887573964496, 3223.44674556213, 3594.0088757396447, 3952.2189349112427, 4310.42899408284, 4668.639053254437, 5026.8491124260345, 5385.059171597633, 5743.2692307692305, 6101.479289940828, 6472.041420118343, 6817.899408284024, 7188.461538461537], [3.2943262673632967, 3.2638600156322126, 3.2518999695748874, 3.2446281622882482, 3.246486083133996, 3.245753135185418, 3.246253934281757, 3.2472569925301102, 3.2356583102522136, 3.2351808720466657, 3.2359284205519883, 3.237169467875278, 3.227800290612279, 3.2273140920726844, 3.2184384136276525, 3.217458716270091, 3.1992065602836877, 3.177878797019038, 2.8807910485472883, 2.179051790010771], Simulation(BattMo.run_battery, LithiumIonBattery("Setup object for a P2D lithium-ion model", {
    "RampUp" => "Sinusoidal"
    "Metadata" =>     {
        "Description" => "Default model settings f

We have a handy function to check parameter, values and bounds:

In [14]:
print_calibration_overview(calibration_05)

NegativeElectrode: Active calibration parameters
┌──────────────────────────────────────────────────┬───────────────┬────────────────────┐
│                                             Name │ Initial value │             Bounds │
├──────────────────────────────────────────────────┼───────────────┼────────────────────┤
│              ActiveMaterial.MaximumConcentration │         31540 │ 10000.0 - 100000.0 │
│ ActiveMaterial.StoichiometricCoefficientAtSOC100 │      0.518738 │          0.0 - 1.0 │
│   ActiveMaterial.StoichiometricCoefficientAtSOC0 │         0.001 │          0.0 - 1.0 │
└──────────────────────────────────────────────────┴───────────────┴────────────────────┘
PositiveElectrode: Active calibration parameters
┌──────────────────────────────────────────────────┬───────────────┬────────────────────┐
│                                             Name │ Initial value │             Bounds │
├──────────────────────────────────────────────────┼───────────────┼────────────────────┤
│ 

### Solve the low-rate calibration

Solving the calibration problem is essentially an optimization problem. We adjust free parameters so to minimize the difference between a target (the data) and the prediction (the simulation result): is performed by solving the optimization problem. This makes use of the adjoint method implemented in Jutul.jl and the LBFGS algorithm.

For calibration, we minimize the squared difference between the predicted and observed voltage, summed over all time steps:  
                  $\sum_i (V_i - V_{exp,i})^2$  
where $V_i$ is the voltage from the model and $V_{exp,i}$ is the voltage from the experimental data at step $i$. This minimization uses in the background cool algorithms implemented in Jutul, the simulation engine of BattMo. 

In [16]:
solve(calibration_05);
cell_parameters_calibrated = calibration_05.calibrated_cell_parameters;


Calibration: Starting calibration of 6 parameters.
It:   0 | val: 3.886e-02 | ls-its: NaN | pgrad: 6.480e+00
It:   1 | val: 1.829e-02 | ls-its: 1 | pgrad: 6.480e+00
It:   2 | val: 4.321e-03 | ls-its: 4 | pgrad: 1.097e+00
It:   3 | val: 4.278e-03 | ls-its: 2 | pgrad: 1.386e-01
It:   4 | val: 4.275e-03 | ls-its: 1 | pgrad: 2.249e-02
It:   5 | val: 4.273e-03 | ls-its: 1 | pgrad: 1.506e-02
It:   6 | val: 4.261e-03 | ls-its: 1 | pgrad: 1.534e-02
It:   7 | val: 4.242e-03 | ls-its: 1 | pgrad: 5.334e-02
It:   8 | val: 4.089e-03 | ls-its: 1 | pgrad: 7.754e-02
LBFGS: Line search unable to succeed in 5 iterations ...
LBFGS: Hessian not updated during iteration 9
It:   9 | val: 3.073e-03 | ls-its: 5 | pgrad: 2.324e-01
LBFGS: Line search unable to succeed in 5 iterations ...
LBFGS: Hessian not updated during iteration 10
It:  10 | val: 3.073e-03 | ls-its: 5 | pgrad: 2.898e-01
Calibration: Calibration finished in 33.8901493 seconds.


We can use the same printing function to explore the results of the simulation

In [17]:
print_calibration_overview(calibration_05)

NegativeElectrode: Active calibration parameters
┌──────────────────────────────────────────────────┬───────────────┬────────────────────┬─────────────────┬──────────┐
│                                             Name │ Initial value │             Bounds │ Optimized value │   Change │
├──────────────────────────────────────────────────┼───────────────┼────────────────────┼─────────────────┼──────────┤
│              ActiveMaterial.MaximumConcentration │         31540 │ 10000.0 - 100000.0 │         22039.3 │  -30.12% │
│ ActiveMaterial.StoichiometricCoefficientAtSOC100 │      0.518738 │          0.0 - 1.0 │        0.546901 │    5.43% │
│   ActiveMaterial.StoichiometricCoefficientAtSOC0 │         0.001 │          0.0 - 1.0 │       0.0263285 │ 2532.85% │
└──────────────────────────────────────────────────┴───────────────┴────────────────────┴─────────────────┴──────────┘
PositiveElectrode: Active calibration parameters
┌──────────────────────────────────────────────────┬───────────────┬─

### Compare the results of the calibration against the experimental data

We can now use the optimized parameters to run a new simulation, and compare the results to the experimental data for the 0.5C discharge curve.

In [19]:
#Setup and run simulation
sim_opt = Simulation(model_setup, cell_parameters_calibrated, cycling_protocol)
output_opt = solve(sim_opt);

#Get simulation data
time_series_opt = get_output_time_series(output_opt)
t_opt = time_series_opt[:Time]
V_opt = time_series_opt[:Voltage]

#Plot
fig = Figure()
ax = Axis(fig[1, 1], title = "CRate = 0.5")
lines!(ax, t0, V0, label = "BattMo initial")
lines!(ax, t_exp_05, V_exp_05, label = "Experimental data")
lines!(ax, t_opt, V_opt, label = "BattMo calibrated", linestyle = :dash)
axislegend(position = :lb)
fig

✔️ Validation of CellParameters passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SimulationSettings passed: No issues found.
──────────────────────────────────────────────────
Jutul: Simulating 2 hours, 12 minutes as 163 report steps


Progress  27%|████████████                               |  ETA: 0:00:00

Progress  55%|████████████████████████                   |  ETA: 0:00:00

Progress  84%|████████████████████████████████████       |  ETA: 0:00:00

Progress 100%|███████████████████████████████████████████| Time: 0:00:00


╭────────────────┬───────────┬───────────────┬──────────╮
│ Iteration type │  Avg/step │  Avg/ministep │    Total │
│                │ 145 steps │ 145 ministeps │ (wasted) │
├────────────────┼───────────┼───────────────┼──────────┤
│ Newton         │   2.13103 │       2.13103 │  309 (0) │
│ Linearization  │   3.13103 │       3.13103 │  454 (0) │
│ Linear solver  │   2.13103 │       2.13103 │  309 (0) │
│ Precond apply  │       0.0 │           0.0 │    0 (0) │
╰────────────────┴───────────┴───────────────┴──────────╯
╭───────────────┬────────┬────────────┬──────────╮
│ Timing type   │   Each │   Relative │    Total │
│               │     ms │ Percentage │       ms │
├───────────────┼────────┼────────────┼──────────┤
│ Properties    │ 0.2413 │    19.50 % │  74.5686 │
│ Equations     │ 0.1888 │    22.41 % │  85.6968 │
│ Assembly      │ 0.0735 │     8.72 % │  33.3660 │
│ Linear solve  │ 0.3852 │    31.13 % │ 119.0383 │
│ Linear setup  │ 0.0000 │     0.00 % │   0.0000 │
│ Precond apply │ 0

### Set up the second calibration

The second calibration is performed against the 2.0C discharge curve. In the same manner as for the first discharge curve, we set up a set of parameters to calibrate against experimental data. The parameters are:

 - The reaction rate constant of both electrodes
 - The diffusion coefficient of both electrodes

The calibration this time starts from the parameters calibrated in the first step, so we use the `cell_parameters_calibrated` from the first `solve` call when defining the new object:

In [ ]:
#Experimental data
t_exp_2 = df_2[:,1]
V_exp_2 = df_2[:,2]

#Update cycling protocol to run at 2C
cycling_protocol2 = deepcopy(cycling_protocol)
cycling_protocol2["DRate"] = 2.0

#Solve simulation
sim2 = Simulation(model_setup, cell_parameters_calibrated, cycling_protocol2)
output2 = solve(sim2);

#Get simulation data
time_series_2 = get_output_time_series(output2)
t2 = time_series_2[:Time]
V2 = time_series_2[:Voltage]

In [ ]:
sim2_0 = Simulation(model_setup, cell_parameters, cycling_protocol2)
output2_0 = solve(sim2_0);
time_series_2_0 = get_output_time_series(output2_0)

t2_0 = time_series_2_0[:Time]
V2_0 = time_series_2_0[:Voltage]

In [ ]:
calibration_2 = VoltageCalibration(t_exp_2, V_exp_2, sim2)

free_calibration_parameter!(calibration_2,
    ["NegativeElectrode","ActiveMaterial", "ReactionRateConstant"];
    lower_bound = 1e-16, upper_bound = 1e-10)
free_calibration_parameter!(calibration_2,
    ["PositiveElectrode","ActiveMaterial", "ReactionRateConstant"];
    lower_bound = 1e-16, upper_bound = 1e-10)

free_calibration_parameter!(calibration_2,
    ["NegativeElectrode","ActiveMaterial", "DiffusionCoefficient"];
    lower_bound = 1e-16, upper_bound = 1e-12)
free_calibration_parameter!(calibration_2,
    ["PositiveElectrode","ActiveMaterial", "DiffusionCoefficient"];
    lower_bound = 1e-16, upper_bound = 1e-12)

print_calibration_overview(calibration_2)

### Solve the second calibration problem

In [ ]:
cell_parameters_calibrated2, = solve(calibration_2);
print_calibration_overview(calibration_2)

### Compare the results of the second calibration against the experimental data

We can now compare the results of the calibrated model against the experimental data for the 2.0C discharge curve. We compare three simulations against the experimental data:
 1. The initial simulation with the original parameters.
 2. The simulation with the parameters calibrated against the 0.5C discharge curve.
 3. The simulation with the parameters calibrated against the 0.5C and 2.0C discharge curves.

In [ ]:
sim_c2 = Simulation(model_setup, cell_parameters_calibrated2, cycling_protocol2)
output2_c = solve(sim_c2, accept_invalid = false);

time_series_2_c = get_output_time_series(output2_c)
t2_c = time_series_2_c[:Time]
V2_c = time_series_2_c[:Voltage]

fig = Figure()
ax = Axis(fig[1, 1], title = "CRate = 2.0")
lines!(ax, t2_0, V2_0, label = "BattMo.jl")
lines!(ax, t2, V2, label = "BattMo.jl (after CRate=0.5 calibration)")

lines!(ax, t_exp_2, V_exp_2, label = "Experimental data")
lines!(ax, t2_c, V2_c, label = "BattMo.jl (after CRate=0.5 + Crate=2.0 calibration)", linestyle = :dash)
axislegend(position = :lb)
fig

### Compare the results of the calibrated model against the experimental data

We can now compare the results of the calibrated model against the experimental data for the 0.5C, 1.0C, and 2.0C discharge curves. Note that we did not calibrate the model for the 1.0C discharge curve, but we still obtain a good fit.

In [ ]:
CRates = [0.5, 1.0, 2.0]
outputs_base = []
outputs_calibrated = []

for CRate in CRates
	cycling_protocol["DRate"] = CRate
	simuc = Simulation(model_setup, cell_parameters, cycling_protocol)

	output = solve(simuc, info_level = -1)
	push!(outputs_base, (CRate = CRate, output = output))

    simc = Simulation(model_setup, cell_parameters_calibrated2, cycling_protocol)
	output_c = solve(simc, info_level = -1)

    push!(outputs_calibrated, (CRate = CRate, output = output_c))
end

colors = Makie.wong_colors()

fig = Figure(size = (1200, 600))
ax = Axis(fig[1, 1], ylabel = "Voltage / V", xlabel = "Time / s", title = "Discharge curve")

for (i, data) in enumerate(outputs_base)
    t_i, V_i = get_tV(data.output)
    lines!(ax, t_i, V_i, label = "Simulation (initial) $(round(data.CRate, digits = 2))", color = colors[i])
end

for (i, data) in enumerate(outputs_calibrated)
    t_i, V_i = get_tV(data.output)
	lines!(ax, t_i, V_i, label = "Simulation (calibrated) $(round(data.CRate, digits = 2))", color = colors[i], linestyle = :dash)
end

for (i, df) in enumerate(dfs)
    t_i, V_i = get_tV(df)
    label = "Experimental $(round(CRates[i], digits = 2))"
	lines!(ax, t_i, V_i, linestyle = :dot, label = label, color = colors[i])
end

fig[1, 2] = Legend(fig, ax, "C rate", framevisible = false)
fig